In [14]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers, models

In [16]:
train_data_path = 'Facial_Images/train' 
val_data_path = 'Facial_Images/validation'

In [18]:
import os
for expression in os.listdir(train_data_path):
    print(expression)


angry
disgust
fear
happy
neutral
sad
surprise
